In [1]:
#r "nuget: Microsoft.ML"
open Microsoft.ML
open Microsoft.ML.Data
open System.Collections.Generic
open System.IO
open System.Net

Installed package Microsoft.ML version *

Create classes for loading and mapping data:

In [2]:
[<CLIMutable>]
type AdultData =
    {
        [<LoadColumn(0)>]
        Age : float32

        [<LoadColumn(1)>]
        WorkClass : string

        [<LoadColumn(2)>]
        Fnlwgt : float32

        [<LoadColumn(3)>]
        Education : string

        [<LoadColumn(4)>]
        EducationNum : float32

        [<LoadColumn(5)>]
        MaritalStatus : float32

        [<LoadColumn(6)>]
        Occupation : float32

        [<LoadColumn(7)>]
        Relationship : float32

        [<LoadColumn(8)>]
        Race : string

        [<LoadColumn(9)>]
        Sex : string

        [<LoadColumn(10)>]
        CapitalGain : float32

        [<LoadColumn(11)>]
        CapitalLoss : float32

        [<LoadColumn(12)>]
        HoursPerWeek : float32

        [<LoadColumn(13)>]
        NativeCountry : string

        [<LoadColumn(14)>]
        [<ColumnName("Label")>]
        Target : string
    }

[<CLIMutable>]
type EncodedLabel =
    {
        Label : string
        EncodedLabel : bool
    }

Download data from [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/machine-learning-databases/adult):

In [3]:
if not <| File.Exists("adult.data") then
    use client = new WebClient()
    client.DownloadFile("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", "adult.data")

Instantiate a new MLContext:

In [4]:
let context = new MLContext()

Load data from file:

In [5]:
let dataView = context.Data.LoadFromTextFile<AdultData>("adult.data", hasHeader = false, separatorChar = ',')

Create our mapping from labels to values:

In [12]:
let labelLookup =
    [|
        KeyValuePair("<=50K", false)
        KeyValuePair("<=50K.", false)
        KeyValuePair(">50K", true)
        KeyValuePair(">50K.", true)
    |]
labelLookup

index,Key,Value
0,<=50K,False
1,<=50K.,False
2,>50K,True
3,>50K.,True


Create an encoder passing in the label-value mappings:

In [7]:
let encoder = context.Transforms.Conversion.MapValue(inputColumnName = "Label", outputColumnName = "EncodedLabel", keyValuePairs = labelLookup)

Fit the encoder to the data:

In [8]:
let transformer = encoder.Fit(dataView)

Transform the data using the encoder:

In [9]:
let transformedDataView = transformer.Transform(dataView)

Print our transformed data:

In [11]:
context.Data.CreateEnumerable<EncodedLabel>(transformedDataView, reuseRowObject = true)
|> Seq.take 10

index,Label,EncodedLabel
0,<=50K,False
1,<=50K,False
2,<=50K,False
3,<=50K,False
4,<=50K,False
5,<=50K,False
6,<=50K,False
7,>50K,True
8,>50K,True
9,>50K,True
